In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%load_ext version_information
%version_information numpy, matplotlib, bayesianchangepoint

Software versions
Python 3.7.3 64bit [Clang 10.0.1 (clang-1001.0.46.3)]
IPython 7.5.0
OS Darwin 18.5.0 x86_64 i386 64bit
numpy 1.16.3
matplotlib 3.0.3
bayesianchangepoint 0.1
Thu May 09 17:36:08 2019 CEST

In [3]:
import os
if not os.path.exists('figures/BCP'):
    os.makedirs('figures/BCP')


# the switching binomial generative model


In [4]:
t_titre = 35
t_label = 25
fig_width= 15
N_scan = 100
#N_scan = 10 # DEBUG

In [5]:
msfigpath = '../PasturelMontagniniPerrinet2019/figures'
observer = 'AM'
mode = 'pari'
exts = ['.png', '.svg', '.pdf']
dpi = 400
from aSPEM import Analysis
e = Analysis(observer=observer, mode=mode)

/!\ Le fichier Delete n'existe pas pour fct_velocity_2_step_False_whitening !


# the online BCP model: figures for the model

In [6]:
N_trials = 100
seed = 2020
np.random.seed(seed)

trials = np.arange(N_trials)
p = np.random.rand(N_trials, 1, 3)
p[:, :, 2] = 0
p[N_trials//2, 0, 2] = 1

p[:N_trials//2, :, 1] = .15
p[N_trials//2:, :, 1] = .85

for trial in trials:
    p[trial, :, 0] =  p[trial, :, 1] > np.random.rand() # Bernouilli trial

In [7]:
N_trials_show = 10
print(p[:N_trials_show, 0, :])

[[0.   0.15 0.  ]
 [0.   0.15 0.  ]
 [0.   0.15 0.  ]
 [0.   0.15 0.  ]
 [0.   0.15 0.  ]
 [0.   0.15 0.  ]
 [1.   0.15 0.  ]
 [1.   0.15 0.  ]
 [1.   0.15 0.  ]
 [0.   0.15 0.  ]]


In [8]:
import bayesianchangepoint as bcp
tau = N_trials/5.
h = 1/tau
i_block = 0

p0, r0 =  0.5, 1.0
p_bar, r_bar, beliefs = bcp.inference(p[:, i_block, 0], h=h, p0=p0, r0=r0)
p_hat, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode='max', p0=p0)

In [9]:
fig, axs = bcp.plot_inference(p[:, i_block, 0], p[:, i_block, 1], p_bar, r, beliefs, eps=1e-18, p0=p0, max_run_length=100)
plt.show()

NameError: name 'r' is not defined

In [ ]:
fig_width = 5
r_cir = 0.1
s_arrow = 0.06
alpha = .4

c_f1 = 'k'
c_f2 = 'r'
c_cir = 'k'
labelsize = 18

layer = 4

NUM = 6


from matplotlib.patches import Circle, Wedge
from matplotlib.collections import PatchCollection


def arrow_1(ax, x, y, alpha=None):
    if alpha is None:
        plt.arrow(x+r_cir, y+r_cir, 1-(r_cir*2)-(s_arrow/2), 1-(r_cir*2)-(s_arrow/2),
                  head_width=s_arrow, head_length=s_arrow, fc=c_f1, ec=c_f1)
    else:
        plt.arrow(x+r_cir, y+r_cir, 1-(r_cir*2)-(s_arrow/2), 1-(r_cir*2)-(s_arrow/2),
                  head_width=s_arrow, head_length=s_arrow, fc=c_f1, ec=c_f1, alpha=alpha)


def arrow_2(ax, x, y, alpha=None):
    if alpha is None:
        if y == 0:
            plt.arrow(x+r_cir, y, 1-(r_cir*2), -y,
                      head_width=0., head_length=0., ec=c_f2)
        else:
            plt.arrow(x+r_cir, y-r_cir, 1-(r_cir*2), -y+r_cir,
                      head_width=0., head_length=0., ec=c_f2)
    else:
        if y == 0:
            plt.arrow(x+r_cir, y, 1-(r_cir*2), -y,
                      head_width=0., head_length=0., ec=c_f2, alpha=alpha)
        else:
            plt.arrow(x+r_cir, y-r_cir, 1-(r_cir*2), -y+r_cir,
                      head_width=0., head_length=0., ec=c_f2, alpha=alpha)


for layer in range(1, 6):
    num = NUM

    if layer == 1:
        num = 2
    if layer == 2:
        num = 3
    if layer == 3:
        num = 4

    fig, ax = plt.subplots(1, 1, figsize=(
        fig_width, fig_width), sharex=True)

    patches = []
    patches2 = []

    if layer < 4:
        trials = range(num)
    else:
        trials = range(int(num*1.6180)+1)

    for x in trials:
        if x < num:
            runlengths = range(x+1)
            lalay = num-1
        else:
            runlengths = range(num)
            lalay = lala[-1]
            
        for r in runlengths:
            if num > 4:
                if y < lalay and x < int(num*1.6180):
                    patches += [Wedge((x, r), r_cir, 0, 360, width=r_cir/10)]
                else:
                    patches2 += [Wedge((x, r), r_cir, 0, 360, width=r_cir/10)]

                if x < int(num*1.6180):
                    if r <= lalay-1:
                        if y != lalay-1 and x < int(num*1.6180)-1:
                            arrow_1(ax, x, r)
                        else:
                            arrow_1(ax, x, r, alpha=alpha)

                        if x < int(num*1.6180)-1:
                            arrow_2(ax, x, r)
                        else:
                            arrow_2(ax, x, r, alpha=alpha)

                    else:
                        arrow_2(ax, x, r, alpha=alpha)
                plt.vlines(9, -0.3, 6.3, color='r', lw=21, alpha=0.01)
            else:
                patches += [Wedge((x, r), r_cir, 0, 360, width=r_cir/10)]
                if r <= lalay-1 and x <= lalay-1:
                    arrow_1(ax, x, r)
                    arrow_2(ax, x, r)

    p = PatchCollection(patches, color=c_cir, lw=1)
    ax.add_collection(p)
    p = PatchCollection(patches2, color=c_cir, alpha=alpha, lw=1)
    ax.add_collection(p)

    plt.tick_params(bottom=True, top=False, right=False, left=True)
    plt.ylabel('Run length', fontsize=labelsize)
    plt.xlabel('Trials', fontsize=labelsize)
    ax.set_xticks(np.arange((NUM+0.32)*1.6180)-1)
    ax.set_xticklabels(np.arange((NUM+0.32)*1.6180).astype(np.int))

    if layer > 4:
        plt.plot([0, 1, 2, 3, 4, 0, 1, 2, 3, 4], lw=10, alpha=0.7, c='k')

    plt.axis([-0.32, (NUM+0.32)*1.6180 -1, -0.32, NUM+0.32])
    for ext in exts: plt.savefig('figures/BCP/bcp_model_layer_%s%s' % (layer, ext))

    plt.show()

In [ ]:
NUM

In [ ]:
figname = '3_BCP_model'
fig.set_figwidth = 5
fig.set_figheight = 5
for ext in exts: fig.savefig(f'{msfigpath}/{figname}' + ext, dpi=dpi, bbox_inches='tight', pad_inches=0)

In [ ]:
from IPython.display import Image
Image(f'{msfigpath}/{figname}.png')

# chosen readout with a comparison with 'fixed'

In [ ]:
figname = '3_BCP_readouts'

In [ ]:
fig, ax1, ax2 = e.plot_bcp(mode=['mean'], mode_compare='fixed', block=1, trial=50, max_run_length=85, 
                 fig_width=8, t_label=25, show_title=None, show_trial=False, N_scan=0)
for card in ['bottom', 'top']: 
    ax1.spines[card].set_visible(False)
    ax2.spines[card].set_visible(False)
#ax1.legend(loc='best')
ax2.set_ylim(0, 80)
     
for ext in exts: fig.savefig(f'{msfigpath}/{figname}' + ext, dpi=dpi, bbox_inches='tight', pad_inches=0)

In [ ]:
from IPython.display import Image
Image(f'{msfigpath}/{figname}.png')

# evaluating the best hazard rate using a likelihood score

Vérifier tau !

In [ ]:
fig, ax1, ax2  = e.plot_bcp(show_trial=False, N_scan=0, block=2, trial=50)
for ext in exts:
    fig.savefig('figures/BCP/BCP_readouts' + ext, dpi=dpi)

In [ ]:
fig = e.plot_bcp(show_trial=False, N_scan=N_scan)
for ext in exts: fig.savefig('figures/BCP/BCP_scan' + ext, dpi=dpi)

In [ ]:
fig = e.plot_bcp(show_trial=False, N_scan=N_scan, pause=True)
#for ext in exts: fig.savefig('figures/BCP/BCP_scan' + ext, dpi=dpi)

TODO: plot pout chaque r la distriubtion Beta correspondant à $P(p |r) = B(p; p(r)*r, (1-p(r))*r)$

In [ ]:
%ls {msfigpath}